# First Attempt at Match Prediction

First attempt at predicting match outcomes for European Soccer data

The focus for simplicity will be on the EPL with the main aim to create the functions required to engineer the features

## 1. Imports



In [1]:
import os

In [2]:
os.environ['PYTHONPATH']

KeyError: 'PYTHONPATH'